Nina Odoux

# **NOTEBOOK 3**

# Este cuaderno se enfocara en :

1. Codificacion de variables categoricas
2. Escalado de variables

In [69]:
import pandas as pd 
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
from category_encoders import TargetEncoder
import os

#importar funciones de mi archivo ipynb:
import sys
sys.path.append("../src")  
import eda_utils as eda

pd.set_option('display.max_columns', 123)
pd.set_option('display.max_rows', 5000)
seed = 123

In [71]:
print(os.getcwd())
os.chdir("C:/Users/ninao/Desktop/ML_practica1EDA")

C:\Users\ninao\Desktop\ML_practica1EDA


In [73]:
# Pour recharger plus tard :
data_train = pd.read_csv("data/processed_data/processed_data_input/data_train_preprocessing_missing_outlier.csv", low_memory=False)
data_test = pd.read_csv("data/processed_data/processed_data_input/data_test_preprocessing_missing_outlier.csv",low_memory=False)

In [75]:
data_train.columns

Index(['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR',
       'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       ...
       'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21',
       'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY',
       'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON',
       'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'TARGET'],
      dtype='object', length=118)

## Codificación de la variable objetivo* class 
1--

* class  0--
LA VARIABLE OBJETIVO YA ESTA EN TIPO INTEGER,

CON DOS CLASES UNICAS, CODIFICACION NO ES NECESARIA
d

In [78]:
# Convertir TARGET a tipo categórico
data_train['TARGET'] = data_train['TARGET'].astype('category')

# Verificar las categorías presentes
print("Categorías en 'TARGET':")
print(data_train['TARGET'].cat.categories)

# Revisar el conteo de cada categoría
print("\nDistribución de 'TARGET':")
print(data_train['TARGET'].value_counts())


Categorías en 'TARGET':
Int64Index([0, 1], dtype='int64')

Distribución de 'TARGET':
0    226148
1     19860
Name: TARGET, dtype: int64


In [80]:
data_train['TARGET'].value_counts()

0    226148
1     19860
Name: TARGET, dtype: int64

In [82]:
X_train = data_train.drop('TARGET',axis=1)
X_test = data_test.drop('TARGET',axis=1)
y_train = data_train['TARGET']
y_test = data_train['TARGET']

In [84]:
categorical_vars, continuous_vars = eda.dame_variables_categoricas(data_train)

#  Codificación del resto de variables categoricas


Por ejemplo, eligo codificarlas con TARGET_encoding todas. Se podría elegir codificar algunas de una forma y otras de otra. Por ejemplo, variables con pocas categorías con one hot encoding y variables con más categorias con TargetEncoding

### Eligo un TARGET Encoding para todas por de momento, puedo pensar en dividir en OneHOT para las variables que tengan menos de 9-10 valores unicos


Porque mis variables categoricas tienen al maximo 20 valores unicos, que puede ser determinado con una baja cardinalidad y que el OneHOT puede gestionar bien

* Aunque la cardinalidad es baja, elige este modelo porque el dataset es muy pesado y no queria anadir mas variables :

* Permite reducir el número de columnas adicionales en comparación con One-Hot Encoding, el TARGET es en correlacion directa con la variable objetivo, y se calculan valores en funcion de su media.


In [88]:
from category_encoders import TargetEncoder
import pandas as pd

# Colonne cible
target_column = 'TARGET'

# 1. Conversion des colonnes catégoriques à 'object' (pour éviter les restrictions)
data_train[categorical_vars] = data_train[categorical_vars].astype('object')

# 2. Remplir les valeurs manquantes
data_train[categorical_vars] = data_train[categorical_vars].fillna('Desconocido')
data_train[target_column] = data_train[target_column].fillna(data_train[target_column].mean())

# 3. Initialisation du TargetEncoder
encoder = TargetEncoder(cols=categorical_vars)

# 4. Encodage sur les colonnes catégoriques de data_train
data_train[categorical_vars] = encoder.fit_transform(data_train[categorical_vars], data_train[target_column])

# 5. Si data_test existe, appliquer le Target Encoding sur les colonnes communes avec data_train
if 'data_test' in globals():
    # Filtrer les colonnes catégoriques présentes dans data_test
    categorical_vars_test = [col for col in categorical_vars if col in data_test.columns]

    # Conversion à 'object' et gestion des valeurs manquantes
    data_test[categorical_vars_test] = data_test[categorical_vars_test].astype('object')
    data_test[categorical_vars_test] = data_test[categorical_vars_test].fillna('Desconocido')

    # Application de la transformation sur data_test
    data_test[categorical_vars_test] = encoder.transform(data_test[categorical_vars_test])

# Vérification des résultats
print("Datos después del Target Encoding (train):")
print(data_train.head())

if 'data_test' in globals():
    print("\nDatos de prueba después del Target Encoding:")
    print(data_test.head())



Datos después del Target Encoding (train):
   SK_ID_CURR  NAME_CONTRACT_TYPE  CODE_GENDER  FLAG_OWN_CAR  FLAG_OWN_REALTY  \
0      454112            0.083477     0.101670      0.072332         0.079274   
1      276860            0.083477     0.101670      0.085058         0.079274   
2      174331            0.083477     0.069847      0.085058         0.079274   
3      128326            0.083477     0.101670      0.085058         0.079274   
4      237052            0.083477     0.069847      0.085058         0.079274   

   CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  AMT_GOODS_PRICE  \
0      0.077317          135000.0   1061599.5     0.281313         1.051982   
1      0.077317          216000.0    760131.0     0.066521         0.260498   
2      0.077317          247500.0    675000.0    -0.357785         0.370088   
3      0.077317          189000.0    355536.0    -0.607650        -0.725812   
4      0.077317           85500.0     99000.0    -1.194601        -1.188526

# Escalado de las variables
Utilicé este metodo porque hemos aprendido que en algunos algoritmos se puede pedir, aunque no siempre hace falta, tenia que probar como se hace 

In [90]:
# Lista de columnas categóricas
categorical_vars_present = [col for col in categorical_vars if col in data_train.columns]

# asignar valores coded de las columnas categóricas a X_train y X_test:
X_train[categorical_vars_present] = data_train[categorical_vars_present]
if 'data_test' in globals():
    categorical_vars_test = [col for col in categorical_vars if col in data_test.columns]
    X_test[categorical_vars_test] = data_test[categorical_vars_test]

# berificar las columnas presentes en X_train y X_test
print("Columnas en X_train después del Target Encoding:")
print(X_train.head())

#selecciono mis columnas numéricas:
numeric_columns = X_train.select_dtypes(include=['number']).columns

if not numeric_columns.any():
    raise ValueError("No hay columnas numéricas para escalar en X_train.")

# SCALING
scaler = StandardScaler()

# austar el escalador con los datos de TRAIN y escalar:
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train[numeric_columns]),  # Escalar datos de entrenamiento
    columns=numeric_columns,                        # Mantener los nombres de las columnas
    index=X_train.index                             # Mantener los índices originales
)

# transformo los datos de TEST usando los mismos parámetros en un nuevo DF:
if 'data_test' in globals():
    X_test_scaled = pd.DataFrame(
        scaler.transform(X_test[numeric_columns]),  # Escalar datos de prueba
        columns=numeric_columns,                   # Mantener los nombres de las columnas
        index=X_test.index                         # Mantener los índices originales
    )
else:
    X_test_scaled = None

# Verificar la forma de los datasets escalados:
print("\nForma de X_train_scaled:", X_train_scaled.shape)
if X_test_scaled is not None:
    print("Forma de X_test_scaled:", X_test_scaled.shape)

# y_train e y_test permanecen sin cambios!!!
print("\nDistribución de y_train:")
print(y_train.value_counts())


Columnas en X_train después del Target Encoding:
   SK_ID_CURR  NAME_CONTRACT_TYPE  CODE_GENDER  FLAG_OWN_CAR  FLAG_OWN_REALTY  \
0      454112            0.083477     0.101670      0.072332         0.079274   
1      276860            0.083477     0.101670      0.085058         0.079274   
2      174331            0.083477     0.069847      0.085058         0.079274   
3      128326            0.083477     0.101670      0.085058         0.079274   
4      237052            0.083477     0.069847      0.085058         0.079274   

   CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  AMT_GOODS_PRICE  \
0      0.077317          135000.0   1061599.5     0.281313         1.051982   
1      0.077317          216000.0    760131.0     0.066521         0.260498   
2      0.077317          247500.0    675000.0    -0.357785         0.370088   
3      0.077317          189000.0    355536.0    -0.607650        -0.725812   
4      0.077317           85500.0     99000.0    -1.194601        -1.

In [91]:
X_train_scaled.describe()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
count,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,246008.0,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05
mean,-6.014866e-17,-1.837502e-15,2.136613e-16,2.538779e-15,-8.683366e-15,-2.397830e-15,-5.654733e-17,-1.173892e-16,1.920714e-17,1.559136e-17,-3.632914e-15,-6.318570e-16,-1.524902e-15,2.171071e-15,8.248238e-17,2.207340e-16,6.127149e-17,4.912261e-17,2.207160e-16,-1.006136e-16,1.008014e-17,1.0,-1.443088e-15,-3.408494e-15,1.756537e-14,-6.728274e-16,-1.427097e-14,2

# Conclusiones finales

*  ¿Hay algún tipo de clientes más propenso a no devolver un préstamo?



Para concluir, no se puede decir que hay un 'persona' tipo que es un tipo de cliente que es mas propenso a no devolver un prestamo, sino que hay combinaciones y una multitud de factores que pueden influir esta probabilidad.

Los resultados del EDA destacan varios factores clave que contribuyen a la probabilidad de que un cliente incumpla con el pago de un préstamo. Estas observaciones no son solo descriptivas, sino que también ofrecen pistas sobre cómo el contexto económico, laboral y personal influye en el comportamiento financiero de los clientes.

La naturaleza del trabajo, particularmente podría ser un indicador de riesgo significativo. Este patrón también refleja la importancia de considerar los ingresos, y otros criterias como los socioeconómico que refuerzan este opinion. Van de mano con la estructura familiar, el numero de ninos, que también desempeñan un papel significativo; familias con múltiples dependientes o situaciones familiares inestables pueden experimentar una mayor presión financiera, y por eso los desempleados y las quienes están en permiso por maternidad tienen una mayor probabilidad de no devolver porque se sabe que hay perfiles con caracteristicas socio economicas mas sensibles a cambios diarios  en la economia y las costumbras de consumo (y por eso representan un riesgo mayor).

En general, el análisis subraya que la propensión al incumplimiento no depende de un solo factor, sino de una combinación compleja de variables laborales, socioeconómicas, demográficas (y características del préstamo tambien). La estabilidad laboral, el flujo de ingresos, la estructura familiar y el contexto externo aparecen como componentes clave en la evaluación del riesgo. 